In [0]:
!pip3 install torch==1.4.0+cu92 torchvision==0.5.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [112]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from itertools import chain 

ERROR! Session/line number was not unique in database. History logging moved to new session 63


In [0]:
BATCH_SIZE = 500

## transformations
transform = transforms.Compose(
    [transforms.ToTensor()]) 

train_set = torchvision.datasets.FashionMNIST("./data", download=True, transform=
                                                transforms.Compose([transforms.ToTensor()]))
test_set = torchvision.datasets.FashionMNIST("./data", download=True, train=False, transform=
                                               transforms.Compose([transforms.ToTensor()]))  

trainloader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=BATCH_SIZE)
testloader = torch.utils.data.DataLoader(test_set,
                                          batch_size=BATCH_SIZE)

In [0]:
class FashionCNN(nn.Module):
    
    def __init__(self):
        super(FashionCNN, self).__init__()
        
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=50, kernel_size=3, padding=1),
            nn.BatchNorm2d(50),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(in_channels=50, out_channels=100, kernel_size=3),
            nn.BatchNorm2d(100),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.conv_layer3 = nn.Sequential(
            nn.Conv2d(in_channels=100, out_channels=200, kernel_size=3),
            nn.BatchNorm2d(200),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(in_features=200*2*2, out_features = 600)
        self.drop = nn.Dropout2d(0.5)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.conv_layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return F.log_softmax(out, dim=1)

In [199]:
## test the model with 1 batch
model = FashionCNN()
for images, labels in trainloader:
    print("batch size:", images.shape)
    out = model(images)
    print(out.shape)
    break

batch size: torch.Size([500, 1, 28, 28])
torch.Size([500, 10])


In [0]:
learning_rate = 0.002
num_epochs = 6

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = FashionCNN()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## compute accuracy
def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [209]:
for epoch in range(num_epochs):
    train_running_loss = 0
    train_acc = 0

    model = model.train()

    ## training step
    for i, (images, labels) in enumerate(trainloader):
        
        images = images.to(device)
        labels = labels.to(device)

        ## forward + backprop + loss
        logits = model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()

        ## update model params
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(logits, labels, BATCH_SIZE)
    
    model.eval()
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
          %(epoch, train_running_loss / (i + 1), train_acc/(i+1)))

Epoch: 0 | Loss: 0.5242 | Train Accuracy: 80.96
Epoch: 1 | Loss: 0.2952 | Train Accuracy: 89.33
Epoch: 2 | Loss: 0.2524 | Train Accuracy: 90.73
Epoch: 3 | Loss: 0.2164 | Train Accuracy: 92.05
Epoch: 4 | Loss: 0.1897 | Train Accuracy: 93.07
Epoch: 5 | Loss: 0.1744 | Train Accuracy: 93.73


In [210]:
test_acc = 0.0

correct_labels = []
predicted_labels = []
for i, (images, labels) in enumerate(testloader, 0):
    images = images.to(device)
    labels = labels.to(device)
    #returns a tensor array of the correct labels in the test set
    correct_labels.append(labels)

    outputs = model(images)
    predictions = torch.max(outputs, 1)[1].to(device)
    predicted_labels.append(predictions)
    #returns a tensor array of the predicted labels of the test set

    test_acc += get_accuracy(outputs, labels, BATCH_SIZE)
        
print('Test Accuracy: %.2f'%( test_acc/(i + 1)))


Test Accuracy: 90.48


In [211]:
#confusion matrix
#converting the tensor array of the correct labels to a regular list
predictions_l = [predicted_labels[i].tolist() for i in range(len(predicted_labels))]
#converting the tensor array of the predicted labels to a regular list
labels_l = [correct_labels[i].tolist() for i in range(len(correct_labels))]
#print(labels_l)
#chain.from_iterable makes it so lists that have [[x1,x2,x3],[y1,y2,y3]] = [x1,x2,x3,y1,y2,y3]
#Basicly it makes a list of lists into one big list.
predictions_l = list(chain.from_iterable(predictions_l))
labels_l = list(chain.from_iterable(labels_l))
#print(labels_l)

#calculates the confusion matrix with the lists of predicted and correct labels
print("accuracy score ", metrics.accuracy_score(labels_l, predictions_l))
confusion_matrix(labels_l, predictions_l)

print("Classification report for CNN :\n%s\n"
      % (metrics.classification_report(labels_l, predictions_l)))

accuracy score  0.9048
Classification report for CNN :
              precision    recall  f1-score   support

           0       0.85      0.86      0.86      1000
           1       0.98      0.99      0.98      1000
           2       0.86      0.89      0.87      1000
           3       0.85      0.94      0.89      1000
           4       0.78      0.91      0.84      1000
           5       0.95      0.99      0.97      1000
           6       0.87      0.57      0.69      1000
           7       0.98      0.94      0.96      1000
           8       0.99      0.98      0.98      1000
           9       0.97      0.97      0.97      1000

    accuracy                           0.90     10000
   macro avg       0.91      0.90      0.90     10000
weighted avg       0.91      0.90      0.90     10000




In [212]:
from google.colab import drive
drive.mount('/content/gdrive')
model_save_name = 'classifier.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [213]:


# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())



Model's state_dict:
conv_layer1.0.weight 	 torch.Size([50, 1, 3, 3])
conv_layer1.0.bias 	 torch.Size([50])
conv_layer1.1.weight 	 torch.Size([50])
conv_layer1.1.bias 	 torch.Size([50])
conv_layer1.1.running_mean 	 torch.Size([50])
conv_layer1.1.running_var 	 torch.Size([50])
conv_layer1.1.num_batches_tracked 	 torch.Size([])
conv_layer2.0.weight 	 torch.Size([100, 50, 3, 3])
conv_layer2.0.bias 	 torch.Size([100])
conv_layer2.1.weight 	 torch.Size([100])
conv_layer2.1.bias 	 torch.Size([100])
conv_layer2.1.running_mean 	 torch.Size([100])
conv_layer2.1.running_var 	 torch.Size([100])
conv_layer2.1.num_batches_tracked 	 torch.Size([])
conv_layer3.0.weight 	 torch.Size([200, 100, 3, 3])
conv_layer3.0.bias 	 torch.Size([200])
conv_layer3.1.weight 	 torch.Size([200])
conv_layer3.1.bias 	 torch.Size([200])
conv_layer3.1.running_mean 	 torch.Size([200])
conv_layer3.1.running_var 	 torch.Size([200])
conv_layer3.1.num_batches_tracked 	 torch.Size([])
fc1.weight 	 torch.Size([600, 800])
fc1.bias